In [74]:
#!pip install RISparser

In [75]:
#!pip install gensim
#!pip install nltk
#!pip install pprintpp

In [76]:
import os
from RISparser import readris
import gensim
from gensim import models
import pprint
from nltk.stem import WordNetLemmatizer
from pprint import pprint
import nltk
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package wordnet to /Users/geopi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [77]:
from gensim.parsing.preprocessing import STOPWORDS
stopwords = set(STOPWORDS)
my_stopwords = {'approach','paper','article','model','viewpoint','conclusion','discussion','method','introduction','section','framework','based','result','research',
'can','which','from','set','it','we','model','thi','new','use'}
all_stopwords = set.union(stopwords,my_stopwords)
#all_stopwords = frozenset(my_stopwords)

In [78]:
data_folder = '/Users/geopi/Documents/Python_Projects/NLP_LDA_InSySPo/NLP_LDA_InSySPo/data/'
files = os.listdir(data_folder)

In [79]:
texts = []
titles = []
abstracts = []
dois = []
#lem = WordNetLemmatizer()
ps = PorterStemmer()

In [80]:
for ff in files:
    f = data_folder + ff
    fid = open(f,'r',encoding="utf8")
    data = readris(fid)

    for i,entry in enumerate(data):
    
        try:
            title = entry['title']
            titles += [title]
            abstract = entry['abstract']
            abstracts += [abstract]
            doi = entry['doi']
            dois += [doi]
            text = title + ' ' + abstract
            text = gensim.utils.simple_preprocess(text)
            for i,t in enumerate(text):
                text[i] = ps.stem(t)
                #text[i] = lem.lemmatize(t)
            text = [word for word in text if word not in all_stopwords]
            texts += [text]
        except:
            pass



In [81]:
dictionary = gensim.corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in texts]

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=4, workers=4)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.026*"predict" + 0.019*"data" + 0.018*"machin" + 0.017*"learn" + 0.010*"classif" + 0.010*"perform" + 0.009*"classifi" + 0.008*"accuraci" + 0.007*"wa" + 0.007*"select"
Topic: 1 
Words: 0.018*"intellig" + 0.014*"technolog" + 0.014*"develop" + 0.013*"ai" + 0.012*"artifici" + 0.010*"design" + 0.010*"decis" + 0.009*"data" + 0.008*"applic" + 0.007*"support"
Topic: 2 
Words: 0.044*"algorithm" + 0.032*"optim" + 0.026*"problem" + 0.018*"propos" + 0.011*"search" + 0.010*"solut" + 0.010*"perform" + 0.009*"network" + 0.008*"solv" + 0.007*"time"
Topic: 3 
Words: 0.034*"network" + 0.023*"data" + 0.022*"neural" + 0.018*"learn" + 0.014*"artifici" + 0.012*"deep" + 0.011*"intellig" + 0.009*"forecast" + 0.009*"predict" + 0.009*"techniqu"
Topic: 4 
Words: 0.031*"imag" + 0.016*"featur" + 0.014*"propos" + 0.012*"detect" + 0.010*"recognit" + 0.008*"segment" + 0.008*"algorithm" + 0.007*"object" + 0.007*"data" + 0.006*"perform"
Topic: 5 
Words: 0.015*"knowledg" + 0.012*"fuzzi" + 0.010*"inform

In [91]:
teste = 1500
for index, score in sorted(lda_model[bow_corpus[teste]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic ({}): {}".format(score, index, lda_model.print_topic(index, 10)))

print('\nTitle:{}\nAbs: {}\nDoi: {}'.format(titles[teste], abstracts[teste], dois[teste]))
print(texts[teste-2])


Score: 0.44684991240501404	 
Topic (1): 0.018*"intellig" + 0.014*"technolog" + 0.014*"develop" + 0.013*"ai" + 0.012*"artifici" + 0.010*"design" + 0.010*"decis" + 0.009*"data" + 0.008*"applic" + 0.007*"support"

Score: 0.43455541133880615	 
Topic (7): 0.024*"learn" + 0.014*"human" + 0.012*"intellig" + 0.011*"comput" + 0.009*"artifici" + 0.007*"cognit" + 0.007*"interact" + 0.007*"machin" + 0.007*"robot" + 0.006*"game"

Score: 0.10265827924013138	 
Topic (3): 0.034*"network" + 0.023*"data" + 0.022*"neural" + 0.018*"learn" + 0.014*"artifici" + 0.012*"deep" + 0.011*"intellig" + 0.009*"forecast" + 0.009*"predict" + 0.009*"techniqu"

Title:Diffuse large B-cell lymphoma outcome prediction by gene-expression profiling and supervised machine learning
Abs: Usually, a proof of a theorem contains more knowledge than the mere fact that the theorem is true. For instance, to prove that a graph is Hamiltonian it suffices to exhibit a Hamiltonian tour in it; however, this seems to contain more knowledg

TypeError: len() takes exactly one argument (4 given)

In [94]:
print(len(titles),' ',len(abstracts),' ',len(dois),' ',len(texts))

127573   124185   108657   108657


In [83]:
new_doc = 'Piriri pororó'

bow_vector = dictionary.doc2bow(preprocess(new_doc))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

NameError: name 'preprocess' is not defined